In [57]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import os
from shutil import copyfile
import time

In [44]:
# dimensions of our images.
img_width, img_height = 224, 224

top_model_weights_path = 'bottleneck_baseline_model.h5'
image_dir = '/data/calTech/test/unknown'
bn_dir = '/data/calTech/bottlenecks'
epochs = 50
nb_samples = len(os.listdir(image_dir))
batch_size = 16

In [22]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the ResNet50 network
    model = applications.ResNet50(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        image_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features = model.predict_generator(
        generator, 
        nb_samples)
    
    np.save(open('/data/calTech/bottleneck_features_calTech_test.npy', 'wb'),
            bottleneck_features)
    
    return bottleneck_features

In [77]:
def save_bottleneck_features():
    # build the ResNet50 network
    model = applications.ResNet50(include_top=False, weights='imagenet')
    
    start = time.time()
    
    for i, file in enumerate(os.listdir(image_dir)):
        
        if (i + 1) % 100 == 0:
            print(i + 1)
            
        image_path = os.path.join(image_dir, file)
        bn_path = os.path.join(bn_dir, os.path.splitext(file)[0] + '.npy')
    
        img = image.load_img(image_path, target_size=(img_width, img_height))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        features = model.predict(x)
    
        np.save(open(bn_path, 'wb'), features)
        
    print((time.time() - start))

In [76]:
save_bottleneck_features()

17.212547063827515
